In [1]:
import obspy as obs
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# from andbro__querrySeismoData import __querrySeismoData
# from andbro__get_timeaxis import __get_timeaxis
# from andbro__get_stream import __getStream

from streamz.dataframe import PeriodicDataFrame

ModuleNotFoundError: No module named 'streamz'

In [2]:
# import panel as pn
# import altair as alt
# from altair import datum
# import pandas as pd
# from vega_datasets import data
# import datetime as dt

In [4]:
import numpy as np
import panel as pn

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.layouts import column, row

pn.extension()

ModuleNotFoundError: No module named 'panel'

In [5]:
config = {}

## set path for figures
config['outpath_figs'] = "/home/brotzer/Desktop/tmp/"

# config['tbeg'] = obs.UTCDateTime("2022-05-04 09:48")
# config['tend'] = obs.UTCDateTime("2022-05-04 09:51")

config['real_time_delay']  = 60 ## minutes
config['period_to_show'] = 5 ## minutes

now = obs.UTCDateTime.now()
config['tbeg'] = now - config['real_time_delay']*60 - config['period_to_show']*60
config['tend'] = now - config['real_time_delay']*60 

config['update_interval'] = 3 ## seconds

## set stations
config['seeds'] = ["BW.RLAS..BJZ", "BW.ROMY.10.BJZ", "BW.ROMY.10.BJZ"]

config['repository'] = "george"

In [6]:
def __getStream(config, restitute=True):
    """
    
    CONFIG:     config['seeds'] list of seed names
                config['tbeg'] startime as UTCDateTime
                config['tend'] endtime as UTCDateTime
                config['repository'] data repository to call [e.g. george, archive, jane,online]


    st = __getStream(config, restitute=True)

    """

    from andbro__querrySeismoData import __querrySeismoData
    from andbro__empty_trace import __empty_trace
    from obspy import Stream
    from numpy import nan
    
    st = Stream()


    for seed in config['seeds']:
        
        net, sta, loc, cha = seed.split(".")
        
#         print(f"loading {seed}...")
        
        try:
            st0, inv0 = __querrySeismoData(  
                                            seed_id=seed,
                                            starttime=config.get("tbeg")-.5,
                                            endtime=config.get("tend")+.5,
                                            repository=config.get("repository"),
                                            path=None,
                                            restitute=False,
                                            detail=None,
                                            fill_value=nan,
                                            )
            if restitute:
                if cha[-2] == "J":
#                     print(" -> removing sensitivity...")
                    st0.remove_sensitivity(inv0)
                elif cha[-2] == "H":
#                     print(" -> removing response...")
                    st0.remove_response(inventory=inv0, output="VEL", zero_mean=True)

            if len(st0) == 1:
                st += st0
            elif len(st0) > 1:
#                 print(" -> merging stream...")
                st += st0.merge()

        except:
            print(f" -> failed to load {seed}!")
            print(f" -> substituted {seed} with NaN values! ")
            st_empty = Stream()
            st_empty.append(__empty_trace(config, seed))
            st += st_empty
    
#     print("\ncompleted loading")

#     print(" -> trimming stream...")
    st.trim(config['tbeg'], config['tend'])
          
 #     print(" -> demean stream...")   
    st.detrend('demean')
    return st

In [7]:
def __update_data(config):
    
    config['tbeg'] = config['tbeg'] + config['update_interval']
    config['tend'] = config['tend'] + config['update_interval']
    
    st = __getStream(config)
    
    df = pd.DataFrame()
    
    df['timeline'] = st[0].times()
#     df['timeline'] = pd.to_datetime(st[0].times(("utcdatetime")).astype(str), 
#                                     utc=True, 
#                                     format='%Y-%m-%dT%H:%M:%S',
#                                     origin="unix")
    
    try:
        for seed in config['seeds']:
            df[seed] = st.select(id=seed)[0].data
    except:
        print("failure")

    return df

In [9]:
st = __getStream(config, restitute=True)

## make Panel 1

net, sta, loc, cha = "BW.RLAS..BJZ".split('.')
# net, sta, loc, cha = config['seeds'][0].split('.')

panel_1 = figure(width=800, height=200, title=f'{sta}.{cha}')

select_st = st.select(station=sta, channel=cha)

panel_1_cds = ColumnDataSource(data={'x': select_st[0].times(), 'y': select_st[0].data})

panel_1.line('x', 'y', source=panel_1_cds)


## make Panel 2


net, sta, loc, cha = "BW.ROMY.10.BJZ".split('.')
# net, sta, loc, cha = config['seeds'][1].split('.')

panel_2 = figure(width=800, height=200, title=f'{sta}.{cha}')

select_st = st.select(station=sta, channel=cha)

panel_2_cds = ColumnDataSource(data={'x': select_st[0].times(), 'y': select_st[0].data})

panel_2.line('x', 'y', source=panel_2_cds)


 -> failed to load BW.RLAS..BJZ!
 -> substituted BW.RLAS..BJZ with NaN values! 
 -> failed to load BW.ROMY.10.BJZ!
 -> substituted BW.ROMY.10.BJZ with NaN values! 
 -> failed to load BW.ROMY.10.BJZ!
 -> substituted BW.ROMY.10.BJZ with NaN values! 


NameError: name 'figure' is not defined

In [ ]:
def __streaming():

    global config
    
    df = __update_data(config)
    
    panel_1_cds.update(data={'x': df['timeline'], 'y': df["BW.RLAS..BJZ"]})
    
    panel_2_cds.update(data={'x': df['timeline'], 'y': df["BW.ROMY.10.BJZ"]})

    pn.io.push_notebook(bk_dash) # Only needed when running in notebook context

#     del df

    
bk_dash = pn.pane.Bokeh(column(panel_2, panel_1))
# bk_dash = pn.Row(pn.Column(panel_1, panel_1))

## create a periodic callback 
interval = int(1e3*config['update_interval']) ## to miliseconds as int
pn.state.add_periodic_callback(__streaming, period=interval)


## make it executable 
bk_dash.servable(title='ROMY Monitor')
# bk_dash.show()


In [ ]:
# pane.save('test.png')